In [2]:
import torch 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [5]:
# Download training data from open datasets
training_data = datasets.FashionMNIST (
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

# Download test data from open datasets
test_data = datasets.FashionMNIST (
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

In [6]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [7]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.299071  [   64/60000]
loss: 2.286794  [ 6464/60000]
loss: 2.268135  [12864/60000]
loss: 2.266523  [19264/60000]
loss: 2.244680  [25664/60000]
loss: 2.214971  [32064/60000]
loss: 2.223128  [38464/60000]
loss: 2.184622  [44864/60000]
loss: 2.187338  [51264/60000]
loss: 2.155968  [57664/60000]
Test Error: 
 Accuracy: 43.9%, Avg loss: 2.148424 

Epoch 2
-------------------------------
loss: 2.158030  [   64/60000]
loss: 2.151450  [ 6464/60000]
loss: 2.087776  [12864/60000]
loss: 2.107098  [19264/60000]
loss: 2.064495  [25664/60000]
loss: 1.991754  [32064/60000]
loss: 2.020627  [38464/60000]
loss: 1.937480  [44864/60000]
loss: 1.947376  [51264/60000]
loss: 1.876458  [57664/60000]
Test Error: 
 Accuracy: 60.6%, Avg loss: 1.873398 

Epoch 3
-------------------------------
loss: 1.905668  [   64/60000]
loss: 1.882887  [ 6464/60000]
loss: 1.754616  [12864/60000]
loss: 1.796819  [19264/60000]
loss: 1.709214  [25664/60000]
loss: 1.640763  [32064/600

In [12]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [13]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

In [21]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[2][0], test_data[2][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Trouser", Actual: "Trouser"


In [27]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [31]:
import sys
import struct

In [ ]:
%%timeit
z = torch.randn(1,1)
result = torch.matmul(z,z)
del z, result

1.51 μs ± 28.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [38]:
%%timeit
z = torch.randn(10000,10000)
result = torch.matmul(z,z)
del z, result

2.4 s ± 58.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(device)

mps


In [39]:
%%timeit
z = torch.randn(1,1).to(device)
result = torch.matmul(z,z)
del z, result

140 μs ± 2.93 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [40]:
%%timeit
z = torch.randn(10000,10000).to(device)
result = torch.matmul(z,z)
del z, result

861 ms ± 5.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
z = torch.tensor([[1,2], [3,4]])
torch.matmul(z,z)

tensor([[ 7, 10],
        [15, 22]])

In [51]:
z = torch.randn(4,4)
z

tensor([[ 0.8374, -1.8908, -1.7204, -0.1844],
        [-1.1673,  0.2203,  0.0839,  0.9342],
        [-1.7010, -1.1274,  0.5419,  1.1096],
        [ 0.2832,  0.8367,  0.2253, -0.2807]])